 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Laboratoire 1: Outils pour l'algèbre linéaire et l'optimisation
Paul Raynaud

Nom : Mohamed Laghdaf HABIBOULLAH

Lien git du fork : [https://github.com/MohamedLaghdafHABIBOULLAH/MTH8408-Hiv24.git](https://github.com/MohamedLaghdafHABIBOULLAH/MTH8408-Hiv24.git)

### Equations du second degré

In [48]:
# Résoudre une équation du second degré
function solvepoly2(a, b, c)
    if a == 0 && b != 0 
        println("Il y'a une seule solution ", - c/b)
        return -c/b
    elseif a == 0 && b == 0 
        if c == 0
            println("Tous les reels sont solution")
        else
            println("Pas de solutions")
        end
    else
        Δ = b^2 - 4 * a * c 
        if Δ >= 0
            Δ1 = sqrt(Δ)
            println("les 2 solutions sont ", (-b - Δ1)/(2 * a), " et ",  (-b + Δ1)/(2 * a))
        else
            Δ1 = sqrt(Complex(Δ))
            println("les 2 solutions sont ", (-b - Δ1)/(2 * a), " et ",  (-b + Δ1)/(2 * a))     
        end
    end
end

solvepoly2 (generic function with 1 method)

In [49]:
solvepoly2(1, 2, 3)

les 2 solutions sont -1.0 - 1.4142135623730951im et -1.0 + 1.4142135623730951im


### Matrrices définies positives

In [50]:
using LinearAlgebra

In [51]:
## Mineurs principaux dominants
function Test_matrice_definie_positive(A)
    if A != A'
        return false
    end
    n = size(A, 1)
    for i in 1:n
        if det(A[1:i, 1:i]) <= 0
            return false
        end
    end
    return true
end

Test_matrice_definie_positive (generic function with 1 method)

In [52]:
A = [1 2; 1 1]
Test_matrice_definie_positive(A)

false

In [53]:
B = [2 1; 1 2]
Test_matrice_definie_positive(B)

true

In [1]:
using Pkg; 
Pkg.activate("jump")

  Activating project at `~/Downloads/MTH8408-Hiv24-master/lab1/jump`


## JuMP: un langage de modélisation

### JuMP ?
JuMP est le language de modélisation pour l'optimisation natif dans Julia. En d'autres termes, on peut modéliser des problèmes d'optimisation en Julia grâce à ce package.

JuMP fait partie de l'organisation [jump-dev](https://jump.dev/) qui propose un ensemble d'outils concernant l'optimisation.

On pourra trouver beaucoup d'informations et d'exemples dans la documentation de JuMP [jump.dev/JuMP.jl/stable/](https://jump.dev/JuMP.jl/stable/)

### Installer JuMP
Comme on l'a fait pour `LinearAlgebra` il faut installer le package `JuMP` via le *package manager*.

In [2]:
] add JuMP

ErrorException: syntax: unexpected "]"

In [ ]:
#ou alors:
using Pkg
Pkg.add("JuMP")

Ensuite, on peut ajouter `JuMP` a notre environement et commencer à l'utiliser.

In [46]:
using JuMP

### Installer un solveur
Dans cet exemple, on va ajouter [`Ipopt`](https://en.wikipedia.org/wiki/IPOPT) qui est un package qui fournit un solveur pour l'optimisation nonlinéaire.

In [4]:
] add Ipopt

ErrorException: syntax: unexpected "]"

In [5]:
#ou alors:
using Pkg
Pkg.add("Ipopt")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...


  No Changes to `~/Downloads/MTH8408-Hiv24-master/lab1/jump/Project.toml`
  No Changes to `~/Downloads/MTH8408-Hiv24-master/lab1/jump/Manifest.toml`


In [47]:
using Ipopt

In [7]:
#ou alors:
using Pkg
Pkg.add("MathOptInterface")

   Resolving package versions...
  No Changes to `~/Downloads/MTH8408-Hiv24-master/lab1/jump/Project.toml`
  No Changes to `~/Downloads/MTH8408-Hiv24-master/lab1/jump/Manifest.toml`


On installe aussi `MathOptInterface` qui permet de communiquer entre le modèle et le solveur.

In [5]:
using MathOptInterface

### Structure d'un modèle JuMP
Tous les modèles écrits avec JuMP ont la même structure:
- un objet `Model`, c'est l'initialisation de notre modèle.
- un objet `Optimizer`, qui spécifie un solveur.
- `Variables`, les variables du problème
- `Objective`, la fonction objectif
- `Constraints`, les contraintes.

#### Exemple d'un code complet: Optimisation non linéaire
On résout ici un problème d'optimisation linéaire
$$
\min_{x \in R^2} 100 (x_2 - x_1^2)^2 + (1 - x_1)^2\
$$

In [57]:
model = Model(Ipopt.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

On définit les variables.

In [58]:
@variable(model, x1)
@variable(model, x2)

x2

Enfin, on ajoute la fonction objectif.

In [59]:
@NLobjective(model, Min, (x1 - 1)^2 + 100 * (x2 - x1)^2)

Pour résoudre ce problème, on peut utiliser la fonction `optimize`.

In [60]:
JuMP.optimize!(model)

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.0000000e+00 0.00e+00 2.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

Une fois la résolution terminée, on peut vérifier le status de la résolution.

In [61]:
@show JuMP.has_values(model)
@show JuMP.termination_status(model) == MathOptInterface.LOCALLY_SOLVED
@show JuMP.primal_status(model) == MathOptInterface.FEASIBLE_POINT
@show JuMP.dual_status(model) == MathOptInterface.FEASIBLE_POINT

JuMP.has_values(model) = true
JuMP.termination_status(model) == MathOptInterface.LOCALLY_SOLVED = true
JuMP.primal_status(model) == MathOptInterface.FEASIBLE_POINT = true
JuMP.dual_status(model) == MathOptInterface.FEASIBLE_POINT = true


true

On peut également faire une analyse sur la solution obtenue.

In [62]:
@show JuMP.value(x1)              # Old syntax: getvalue(x)
@show JuMP.value(x2)              # Old syntax: getvalue(y)
@show JuMP.objective_value(model)       # Old syntax: getobjectivevalue(model)

JuMP.value(x1) = 0.9999999999999922
JuMP.value(x2) = 0.9999999999999922
JuMP.objective_value(model) = 6.039716305598372e-29


6.039716305598372e-29

## Solve an NLPModel with Ipopt

The problem created with `JuMP` can be solved with `Ipopt`.

In [68]:
Pkg.add("ADNLPModels")

   Resolving package versions...


    Updating `~/Downloads/MTH8408-Hiv24-master/lab1/jump/Project.toml`
  [54578032] + ADNLPModels v0.7.0
    Updating `~/Downloads/MTH8408-Hiv24-master/lab1/jump/Manifest.toml`
  [54578032] + ADNLPModels v0.7.0
  [ffa27691] + ColPack v0.3.0
  [069b7b12] + FunctionWrappers v1.1.3
  [37e2e3b7] + ReverseDiff v1.15.1
  [f218ff0c] + ColPack_jll v0.3.0+0


In [ ]:
Pkg.add("NLPModels")

   Resolving package versions...


    Updating `~/Downloads/MTH8408-Hiv24-master/lab1/nlpmodels/Project.toml`
⌃ [b6b21f68] + Ipopt v1.4.0
  No Changes to `~/Downloads/MTH8408-Hiv24-master/lab1/nlpmodels/Manifest.toml`


In [66]:
Pkg.add("NLPModelsIpopt")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...


    Updating `~/Downloads/MTH8408-Hiv24-master/lab1/jump/Project.toml`
  [f4238b75] + NLPModelsIpopt v0.10.1
    Updating `~/Downloads/MTH8408-Hiv24-master/lab1/jump/Manifest.toml`
  [14f7f29c] + AMD v0.5.3
  [e2ba6199] + ExprTools v0.1.10
  [9aa1b823] + FastClosures v0.3.2
  [40e66cde] + LDLFactorizations v0.10.1
  [5c8ed15e] + LinearOperators v2.6.0
  [a4795742] + NLPModels v0.20.0
  [f4238b75] + NLPModelsIpopt v0.10.1
  [ae029012] + Requires v1.3.0
  [ff4d7338] + SolverCore v0.3.7
  [a759f4b9] + TimerOutputs v0.5.23
  [bea87d4a] + SuiteSparse_jll v5.10.1+0


In [63]:
using JuMP, Ipopt

model = Model(Ipopt.Optimizer)
x0 = [-1.2; 1.0]
@variable(model, x[i=1:2], start=x0[i])
@NLobjective(model, Min, (x[1] - 1)^2 + 100 * (x[2] - x[1]^2)^2)
optimize!(model)

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.4200000e+01 0.00e+00 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

In [75]:
@show JuMP.has_values(model)
@show JuMP.termination_status(model) == MathOptInterface.LOCALLY_SOLVED
@show JuMP.primal_status(model) == MathOptInterface.FEASIBLE_POINT
@show JuMP.dual_status(model) == MathOptInterface.FEASIBLE_POINT

JuMP.has_values(model) = true
JuMP.termination_status(model) == MathOptInterface.LOCALLY_SOLVED = true
JuMP.primal_status(model) == MathOptInterface.FEASIBLE_POINT = true
JuMP.dual_status(model) == MathOptInterface.FEASIBLE_POINT = true


true

The same can be achieved very easily with `NLPModels` using the package `NLPModelsIpopt` and its function `ipopt`.

In [73]:
using NLPModels, NLPModelsIpopt, ADNLPModels

nlp = ADNLPModel(x -> (x[1] - 1)^2 + 100 * (x[2] - x[1]^2)^2, [-1.2; 1.0])
stats = ipopt(nlp)
print(stats)

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.4200000e+01 0.00e+00 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

In [71]:
stats.objective

3.743975643139474e-21

In [72]:
stats.solution  

2-element Vector{Float64}:
 0.9999999999400667
 0.9999999998789006